In [1]:
import pandas as pd
import numpy as np

In [2]:
reviews = pd.read_csv("../buffet_reviews.csv")
rest = pd.read_csv("../buffet_restaurants_merge.csv")
rest = rest[["business_id", "name", "address", "stars"]]
rest = rest.rename(columns={"stars":"rating"})
reviews = rest.merge(reviews, on=["business_id"])
print(len(reviews))
reviews.head(3)

15221


,business_id,name,address,rating,review_id,user_id,stars,useful,funny,cool,text,date
0,gsicNDwrLH4WGeWMDcJv6w,Ahmed Indian Restaurant 1,"11301 S Orange Blossom Trl, Ste 104",3.5,-MAJ1oZNMqWloQrLMZ4P3A,YbLgr0aG1r7bvvTlpyiDrA,4.0,0,1,0,Stopped by today for lunch buffet. I was hopin...,2015-11-06 02:07:30
1,gsicNDwrLH4WGeWMDcJv6w,Ahmed Indian Restaurant 1,"11301 S Orange Blossom Trl, Ste 104",3.5,Rpqj59E5pzwOU1VyUGLzAQ,qwfVavvEHKTR015tTvDsMQ,5.0,0,0,0,We have been to Ahmed indian restaurant and it...,2016-03-07 01:18:14
2,gsicNDwrLH4WGeWMDcJv6w,Ahmed Indian Restaurant 1,"11301 S Orange Blossom Trl, Ste 104",3.5,l_3lauzEQdXOQTFzUke4eA,i6LKe_wPR2DV0ZGGM42ucQ,3.0,0,0,0,"This was my first at an Indian restrestaurant,...",2014-10-11 19:21:16


In [3]:
review_counts = reviews.groupby(["name", "address"],
                as_index=False).agg({"review_id":"count"})
review_counts["groupid"] = "1"
review_counts = review_counts.rename(columns={"name":"id", "review_id":"size"})
# review_counts["size"] = review_counts["size"] / 5
# review_counts.to_csv("../reviews_counts.csv", index=False)
review_counts = review_counts.drop("address", axis=1)
review_counts

,id,size,groupid
0,Ahmed Indian Restaurant 1,148,1
1,Ahmed Indian Restaurant 2,139,1
2,Bachata Breeze Restaurant,9,1
3,Backlot Express,130,1
4,Bamboo Garden Super Buffet,11,1
...,...,...,...
79,Tony's Brazilian Grill,53,1
80,Tropical Cabana,150,1
81,U & Me Revolving Hot Pot,105,1
82,Villa de Flora,144,1


In [4]:
reviews = reviews[reviews["text"].str.len() < 300]

In [5]:
largest = reviews.groupby(['name']).apply(lambda x: x.nlargest(3,['stars'])).reset_index(drop=True)
largest['rank'] = largest.groupby(["name"]).text.transform(lambda x : pd.factorize(x)[0]+1)
largest_n = largest.pivot_table(index=["name", "address", "rating"],
                         columns="rank",values="text",
                          aggfunc=lambda x: ' '.join(x)).reset_index()
largest_n = largest_n.rename(columns={1:"good_one", 2:"good_two", 3:"good_three"})

smallest = reviews.groupby(['name']).apply(lambda x: x.nsmallest(3,['stars'])).reset_index(drop=True)
smallest['rank'] = smallest.groupby(["name"]).text.transform(lambda x : pd.factorize(x)[0]+1)
smallest_n = smallest.pivot_table(index=["name", "address", "rating"],
                         columns="rank",values="text",
                          aggfunc=lambda x: ' '.join(x)).reset_index()
smallest_n = smallest_n.rename(columns={1:"bad_one", 2:"bad_two", 3:"bad_three"})

most_data = largest_n.merge(smallest_n, on=["name", "address", "rating"])
most_data = most_data.rename(columns={"name":"id"})
most_data

rank,id,address,rating,good_one,good_two,good_three,bad_one,bad_two,bad_three
0,Ahmed Indian Restaurant 1,"11301 S Orange Blossom Trl, Ste 104",3.5,We have been to Ahmed indian restaurant and it...,"We tried their lunch buffet, all dishes were c...","The buffet looked too small, ...but it what wa...",Grill items were terrible - damn cook never at...,Disgusting food! They served biriyani made wit...,Worst food ever !!! Don't even think of coming...
1,Ahmed Indian Restaurant 2,10042 University Blvd,4.0,Highly recommended! Try them out. Great servic...,Ordered via uber eats.. very flavorful and tha...,Great place delicious food it couldn't get bet...,If you only eat Pakistani food. Do not eat her...,I can make better Indian food by myself. I pla...,Service is worst. 25% Gratuity automatically a...
2,Bachata Breeze Restaurant,5773 Curry Ford Rd,3.0,"AMAZING FOOD! Simple, delicious, inexpensive L...",For dancing this place is awesome! The food is...,Great price $4.99for a Latin buffet. Good var...,Went In one night and the kitchen was closed b...,My husband and I both got sick eating here. Th...,Great price $4.99for a Latin buffet. Good var...
3,Backlot Express,351 S Studio Dr,3.5,Love it here - SO easy and reasonably priced.....,Great quick stop. Star Wars themed. Good mu...,Quick and convenient. I used mobile order pic...,Overpriced mediocrity at its finest. Disney ta...,When McDonald's serves fresher quality food th...,This place smells really bad on the inside and...
4,Bamboo Garden Super Buffet,3125 Columbia Blvd,2.5,"Was there last night about 630, food was fresh...",This buffet is great. Ate there last Wednesday...,NEVER had a problem here and we eat here once ...,NEVER had a problem here and we eat here once ...,"Was there last night about 630, food was fresh...",This buffet is great. Ate there last Wednesday...
...,...,...,...,...,...,...,...,...,...
79,Tony's Brazilian Grill,5159 International Dr,4.0,We loved this steak house in Orlando. Large an...,"Best Brazilian food in town, staff are very ni...",After eating burgers and fast food at the Disn...,This is the worse food I have ever tasted! If ...,Not worth $$$. Didn't even stay. Save your mon...,A few selection. The food is good in general; ...
80,Tropical Cabana,1875 Central Florida Pkwy,5.0,"Delicious, fresh, well-priced, and incredibly ...","Great service , owner's son is a big Seattle S...",My husband and I have been going here for year...,Don't buy empanadas waste of money they are em...,I am on a quest to try to find a good Cuban sa...,"La comida estaba fría y en extremo grasosa, el..."
81,U & Me Revolving Hot Pot,12384 S Apopka Vineland Rd,4.0,An overall great experience! From Peanut the\n...,this restaurant is great and everything there ...,It is a great place! Very unique in the Orland...,"This is currently reservation only, that's not...","Absolutely awful service, first time at hot po...",it's cool because of the robot.. sat us in the...
82,Villa de Flora,6000 W Osceola Pkwy,3.5,I have a gluten allergy and the chef Deddie fr...,"Amazing dining experience, beautiful atmospher...",Awesome spanish buffet. \nAnd the view and env...,"Ugh, overpriced hotel food. When you are stuc...",I've read that this is the best brunch in Orla...,Wish I could give it less than 1 star complete...


In [6]:
most_data = most_data.merge(review_counts, on="id")
most_data

,id,address,rating,good_one,good_two,good_three,bad_one,bad_two,bad_three,size,groupid
0,Ahmed Indian Restaurant 1,"11301 S Orange Blossom Trl, Ste 104",3.5,We have been to Ahmed indian restaurant and it...,"We tried their lunch buffet, all dishes were c...","The buffet looked too small, ...but it what wa...",Grill items were terrible - damn cook never at...,Disgusting food! They served biriyani made wit...,Worst food ever !!! Don't even think of coming...,148,1
1,Ahmed Indian Restaurant 2,10042 University Blvd,4.0,Highly recommended! Try them out. Great servic...,Ordered via uber eats.. very flavorful and tha...,Great place delicious food it couldn't get bet...,If you only eat Pakistani food. Do not eat her...,I can make better Indian food by myself. I pla...,Service is worst. 25% Gratuity automatically a...,139,1
2,Bachata Breeze Restaurant,5773 Curry Ford Rd,3.0,"AMAZING FOOD! Simple, delicious, inexpensive L...",For dancing this place is awesome! The food is...,Great price $4.99for a Latin buffet. Good var...,Went In one night and the kitchen was closed b...,My husband and I both got sick eating here. Th...,Great price $4.99for a Latin buffet. Good var...,9,1
3,Backlot Express,351 S Studio Dr,3.5,Love it here - SO easy and reasonably priced.....,Great quick stop. Star Wars themed. Good mu...,Quick and convenient. I used mobile order pic...,Overpriced mediocrity at its finest. Disney ta...,When McDonald's serves fresher quality food th...,This place smells really bad on the inside and...,130,1
4,Bamboo Garden Super Buffet,3125 Columbia Blvd,2.5,"Was there last night about 630, food was fresh...",This buffet is great. Ate there last Wednesday...,NEVER had a problem here and we eat here once ...,NEVER had a problem here and we eat here once ...,"Was there last night about 630, food was fresh...",This buffet is great. Ate there last Wednesday...,11,1
...,...,...,...,...,...,...,...,...,...,...,...
79,Tony's Brazilian Grill,5159 International Dr,4.0,We loved this steak house in Orlando. Large an...,"Best Brazilian food in town, staff are very ni...",After eating burgers and fast food at the Disn...,This is the worse food I have ever tasted! If ...,Not worth $$$. Didn't even stay. Save your mon...,A few selection. The food is good in general; ...,53,1
80,Tropical Cabana,1875 Central Florida Pkwy,5.0,"Delicious, fresh, well-priced, and incredibly ...","Great service , owner's son is a big Seattle S...",My husband and I have been going here for year...,Don't buy empanadas waste of money they are em...,I am on a quest to try to find a good Cuban sa...,"La comida estaba fría y en extremo grasosa, el...",150,1
81,U & Me Revolving Hot Pot,12384 S Apopka Vineland Rd,4.0,An overall great experience! From Peanut the\n...,this restaurant is great and everything there ...,It is a great place! Very unique in the Orland...,"This is currently reservation only, that's not...","Absolutely awful service, first time at hot po...",it's cool because of the robot.. sat us in the...,105,1
82,Villa de Flora,6000 W Osceola Pkwy,3.5,I have a gluten allergy and the chef Deddie fr...,"Amazing dining experience, beautiful atmospher...",Awesome spanish buffet. \nAnd the view and env...,"Ugh, overpriced hotel food. When you are stuc...",I've read that this is the best brunch in Orla...,Wish I could give it less than 1 star complete...,144,1


In [7]:
most_data.to_csv("../reviews_data.csv", index=False)